<a href="https://colab.research.google.com/github/hyewwn/COSE474/blob/main/CLIP_tuning_add_augmentation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Set directory

In [1]:
try:
    import google.colab
    from google.colab import drive
    drive.mount('/content/drive', force_remount = True)
    IN_COLAB = True
except:
    IN_COLAB = False

Mounted at /content/drive


In [2]:
import os
import torch

if IN_COLAB:
    root_path = '/content/drive/MyDrive/딥러닝/project/experiment'
else:
    root_path = 'Enter local path'

use_mps = True if torch.has_mps else False
os.chdir(root_path)

<ipython-input-2-b3d1b7935514>:9: UserWarning: 'has_mps' is deprecated, please use 'torch.backends.mps.is_built()'
  use_mps = True if torch.has_mps else False


# PIP install

In [3]:
pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 521.2/521.2 kB 8.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 12.6 MB/s eta 0:00:00


In [4]:
pip install git+https://github.com/openai/CLIP.git

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-5b2s_byf
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-5b2s_byf
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.4/53.4 kB 2.0 MB/s eta 0:00:00
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=ff1b81d72864ed333d54744cb0825184683c9f3fe3acecfe64e5ebff88988d52
  Stored in directory: /tmp/pip-ephem-wheel-cache-98j0_oqc/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip


# Import packages

In [5]:
from transformers import CLIPModel, CLIPTokenizer, CLIPProcessor
from datasets import load_dataset, load_metric, DatasetDict
from torch.optim import AdamW
from tqdm import tqdm
from PIL import Image
import requests
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
from torchvision import transforms
import clip
import torchtext
import numpy as np
import random
from torch.nn import DataParallel
from transformers import AutoTokenizer, AutoModelForSeq2SeqLM
import pandas as pd
from CLIP_AUG.instructions import InstructionsHandler
import warnings
warnings.filterwarnings('ignore')

# Load Dataset

In [6]:
# Load the dataset
dataset = load_dataset("JotDe/mscoco_15k")
dataset

Extracting data files:   0%|          | 0/2 [00:00<?, ?it/s]

Generating train split:   0%|          | 0/9988 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/4994 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['image', 'text'],
        num_rows: 9988
    })
    test: Dataset({
        features: ['image', 'text'],
        num_rows: 4994
    })
})

# Reduce size of dataset

In [7]:
small_train_dataset = dataset['train'].shuffle(seed=42).select([i for i in range(90)])
small_test_dataset = dataset['test'].shuffle(seed=42).select([i for i in range(40)])

# 새로운 DatasetDict 생성
small_dataset_dict = DatasetDict({
    'train': small_train_dataset,
    'test': small_test_dataset
})

# Load models

In [8]:
# Load the CLIP model and tokenizer
model_name = "openai/clip-vit-base-patch32"
model = CLIPModel.from_pretrained(model_name)
tokenizer = CLIPTokenizer.from_pretrained(model_name)
processor = CLIPProcessor.from_pretrained(model_name)

config.json:   0%|          | 0.00/4.19k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/605M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/568 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/862k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/525k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/389 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.22M [00:00<?, ?B/s]

preprocessor_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [10]:
model, preprocess = clip.load("ViT-B/32", device=device, jit=False)

100%|███████████████████████████████████████| 338M/338M [00:06<00:00, 55.4MiB/s]


In [11]:
if device == "cpu":
  model.float()
if device != "cpu":
    model = DataParallel(model)

In [12]:
# Function to convert model's parameters to FP32 format
def convert_models_to_fp32(model):
    for p in model.parameters():
        p.data = p.data.float()
        p.grad.data = p.grad.data.float()

# Dataloader

In [13]:
# Get the input text into the required format using Instructions
instruct_handler = InstructionsHandler()
instruct_handler.load_instruction_set1()
bos_instruction = instruct_handler.aug['bos_instruct_double']
delim_instruct = instruct_handler.aug['delim_instruct']
eos_instruct = instruct_handler.aug['eos_instruct']

In [16]:
class Text2TextDataset(Dataset):
    def __init__(self, dataset, tokenizer, model, bos_instruction, delim_instruct, eos_instruct):
        self.dataset = dataset
        self.tokenizer = tokenizer
        self.model = model
        self.bos_instruction = bos_instruction
        self.delim_instruct = delim_instruct
        self.eos_instruct = eos_instruct

    def __len__(self):
        return len(self.dataset)

    def __getitem__(self, idx):
        # Get image and text from the original dataset
        image = preprocess(self.dataset[idx]['image'])
        original_text = self.dataset[idx]['text']

        # Perform text2text generation
        input_text = self.bos_instruction + original_text + self.delim_instruct + self.eos_instruct
        tokenized_text = self.tokenizer(input_text, return_tensors="pt")
        output_ids = self.model.generate(tokenized_text.input_ids)

        # Decode generated text
        generated_text = self.tokenizer.decode(output_ids[0], skip_special_tokens=True)

        out = {'image': image, 'origin_text':original_text, 'gen_text':generated_text}

        return out

model_checkpoint = 'allenai/tk-instruct-base-def-pos'
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
gen_model = AutoModelForSeq2SeqLM.from_pretrained(model_checkpoint)

# DataLoader 파라미터 정의
batch_size = 100
shuffle = True

dataset_dict = {
    "train": Text2TextDataset(small_dataset_dict['train'], tokenizer, gen_model, bos_instruction, delim_instruct, eos_instruct),
    "test": Text2TextDataset(small_dataset_dict['test'], tokenizer, gen_model, bos_instruction, delim_instruct, eos_instruct)
}

# DataLoader 생성
dataloaders = {split: DataLoader(dataset, batch_size=batch_size, shuffle=shuffle) for split, dataset in dataset_dict.items()}


tokenizer_config.json:   0%|          | 0.00/2.13k [00:00<?, ?B/s]

spiece.model:   0%|          | 0.00/792k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.42M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/1.79k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/677 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/495M [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/147 [00:00<?, ?B/s]

In [17]:
trainloader = dataloaders['train']
testloader = dataloaders['test']

# Train

In [18]:
from torch.nn.functional import log_softmax
from nltk.translate.bleu_score import corpus_bleu

# Prepare the optimizer
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5,betas=(0.9,0.98),eps=1e-6,weight_decay=0.2)
# Specify the loss function
loss_img = nn.CrossEntropyLoss()
loss_txt = nn.CrossEntropyLoss()

best_te_loss = 1e5
best_ep = -1
# Train the model
num_epochs = 3
for epoch in range(num_epochs):
  print(f"running epoch {epoch}, best test loss {best_te_loss} after epoch {best_ep}")
  model.train()
  tr_loss = 0
  step = 0
  pbar = tqdm(trainloader, total=len(trainloader))
  for batch in pbar:
    step += 1
    optimizer.zero_grad()

    images,texts1, texts2 = batch['image'], batch['origin_text'], batch['gen_text']

    images= images.to(device)
    texts1 = clip.tokenize(texts1).to(device)
    texts2 = clip.tokenize(texts2).to(device)

    # Forward pass
    logits_per_image1,logits_per_text1  = model(images, texts1)
    logits_per_image2,logits_per_text2  = model(images, texts2)


    # Compute loss
    ground_truth = torch.arange(len(images),dtype=torch.long,device=device)
    total_loss = (loss_img(logits_per_image1,ground_truth)+ loss_img(logits_per_image2,ground_truth) + loss_txt(logits_per_text1,ground_truth) + loss_txt(logits_per_text2,ground_truth))/4


    # Backward pass
    total_loss.backward()
    tr_loss += total_loss.item()
    if device == "cpu":
      optimizer.step()
    else :
      convert_models_to_fp32(model)
      optimizer.step()
      clip.model.convert_weights(model)

    pbar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item():.4f}", refresh=True)
  tr_loss /= step

  # Eval
  step = 0
  te_loss = 0
  with torch.no_grad():
    model.eval()
    test_pbar = tqdm(testloader, total=len(testloader))
    for batch in test_pbar:
      step += 1
      images,texts = batch['image'], batch['origin_text']
      images= images.to(device)
      texts = clip.tokenize(texts).to(device)
      logits_per_image,logits_per_text  = model(images, texts)
      ground_truth = torch.arange(len(images),dtype=torch.long,device=device)

      total_loss = (loss_img(logits_per_image,ground_truth) + loss_txt(logits_per_text,ground_truth))/2
      te_loss += total_loss.item()
      test_pbar.set_description(f"Epoch {epoch+1}/{num_epochs}, Loss: {total_loss.item():.4f}", refresh=True)
    te_loss /= step

  # Save the model if the test loss is the best we've seen so far.
  if te_loss < best_te_loss:
    best_te_loss = te_loss
    best_ep = epoch
    torch.save(model.state_dict(), "test2_best_model.pt")
    print(f"epoch {epoch+1}, tr_loss {tr_loss}, te_loss {te_loss}")
torch.save(model.state_dict(), "test2_last_model.pt")

running epoch 0, best test loss 100000.0 after epoch -1


Epoch 1/3, Loss: 0.9697: 100%|██████████| 1/1 [00:28<00:00, 28.29s/it]


epoch 1, tr_loss 0.5048828125, te_loss 0.9697265625
running epoch 1, best test loss 0.9697265625 after epoch 0


  0%|          | 0/1 [00:51<?, ?it/s]


KeyboardInterrupt: ignored

# Evaluating Precision on Validation Set



In [ ]:
model.load_state_dict(torch.load("/content/drive/MyDrive/딥러닝/project/experiment/test2_best_model.pt"))
NUM_NEG = 30
NUM_TEST = 400

In [ ]:
n_correct = 0
for i in tqdm(range(NUM_TEST)):
  empty = True
  while empty:
    idx = random.choice(range(len(small_dataset_dict['test'])))
    image = small_dataset_dict['test'][idx]['image']
    image = preprocess(image).unsqueeze(0).to(device)
    texts = small_dataset_dict['test'][idx]['text']
    if len(texts) > 0:
      pos_txt = random.choice(texts)
      empty = False

  neg_i = 0
  neg_txts = []
  while neg_i < NUM_NEG:
    image_idx = random.choice(range(len(small_dataset_dict['test'])))
    neg_idx = image_idx
    if neg_idx == idx:
      continue
    texts = small_dataset_dict['test'][image_idx]['text']
    if len(texts) == 0:
      continue
    neg_txt = random.choice(texts)
    if neg_txt in neg_txts:
      continue
    neg_txts.append(neg_txt)
    neg_i += 1

  text = clip.tokenize([pos_txt] + neg_txts).to(device)

  with torch.no_grad():
    image_features = model.encode_image(image)
    text_features = model.encode_text(text)

    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  if np.argmax(probs) == 0:
    n_correct += 1

print(f"\nPrecision: {n_correct/NUM_TEST}")

# Evaluating BLEU and Word Diverisity using Naive Sampling

## Sampling Captions for Validation Images According to CLIP Text-Image Proximity

In [ ]:
def sample1Caption(img_idx, corpus, model, num_cand):
  image = preprocess(small_dataset_dict['test']['image'][img_idx]).unsqueeze(0).to(device)
  i = 0
  txts = []
  while i < num_cand:
    txt = random.choice(corpus)
    if txt in txts:
      continue
    if len(txt.split()) < 5 or len(txt) >72:
      continue
    txts.append(txt)
    i += 1

  text = clip.tokenize(txts).to(device)

  with torch.no_grad():
    logits_per_image, logits_per_text = model(image, text)
    probs = logits_per_image.softmax(dim=-1).cpu().numpy()

  return txts[np.argmax(probs)]

In [ ]:
corpus = []
for txt in small_dataset_dict['train']['text']:
  txtlist = [txt]
  corpus += txtlist
len(corpus), corpus[0]

In [ ]:
captions = {}
for i in tqdm(range(len(small_dataset_dict['test']))):
  caption = sample1Caption(i, corpus, model, 100)
  captions[i] = caption

In [ ]:
for get_bleu in range(1,4):
  bleu_x_lst = []
  bleu_y_lst = []
  for p, caps in enumerate(small_dataset_dict['test']['text']):
    if not caps:
      continue
    bleu_x_lst.append(captions[p].split())
    splittedcaps = [x.split() for x in caps]
    bleu_y_lst.append(splittedcaps)
  BLEU = torchtext.data.metrics.bleu_score(bleu_x_lst, bleu_y_lst, max_n=get_bleu, weights=[1/get_bleu]*get_bleu)
  print(f"BLEU-{get_bleu}-gram score: {BLEU}")
